In [1]:
import numpy as np
import pandas as pd
from PIL import Image
import time
import os, struct
from array import array as pyarray
from numpy import append, array, int8, uint8, zeros
from pylab import *
from numpy import *
from PIL import Image
import math
import gzip

In [2]:
labels_f = list()
f = gzip.open('train-labels-idx1-ubyte.gz','r')
f.read(8)
for i in range(0,60000):   
    buf = f.read(1)
    labels_t = np.frombuffer(buf, dtype=np.uint8).astype(np.int64)
    labels_f.append(labels_t)



In [3]:
start_time = time.time()



def point_sum(mat , x , y):
    sum  = 0
    a = x
    b = y
    for i in range(a+1):
        for j in range(b+1):
            sum  += mat[i,j]
    return sum


def summed_mat(mat):
    for k in range(mat.shape[0]):
        for l in range(mat.shape[1]):
            integral_mat[k,l] = point_sum(mat,  k, l)
    return integral_mat


def get_area(integral_image, top_left, bottom_right):

    top_left = (top_left[0]-1, top_left[1]-1)
    bottom_right = (bottom_right[0], bottom_right[1])

    top_right = (top_left[0], bottom_right[1])
    bottom_left = (bottom_right[0], top_left[1])

    area = integral_image[top_left] - integral_image[top_right] - integral_image[bottom_left] + integral_image[bottom_right]
    return area


In [4]:
def enum(**enums):
    return type('Enum', (), enums)


feature = enum(Two_horz = (1,2), Two_vert = (2,1))
feature_type = [feature.Two_horz, feature.Two_vert]
frame_size = 24

def haar_feature_value(integral_image, type_of_feature, start_position, width, height):

    haar_value = 0
    start_position = (start_position[0], start_position[1])


    if type_of_feature == feature.Two_horz:
        bright = get_area(integral_image_done, start_position, (start_position[0]+int(width/2), start_position[1]+height))
        dark   = get_area(integral_image_done, (start_position[0]+int(width/2), height), (start_position[0]+width, start_position[1]+height))
        haar_value = bright - dark

    elif type_of_feature == feature.Two_vert:
        bright = get_area(integral_image_done, start_position, (start_position[0]+width, start_position[1]+int(height/2)))
        dark   = get_area(integral_image_done, (start_position[0], start_position[1]+int(height/2)), (start_position[0]+width, start_position[1]+height))
        haar_value = bright - dark

    return haar_value


def get_vote(Haar_value, polarity, threshold):
    return 1 if Haar_value < polarity * threshold else -1

In [5]:
data = pd.DataFrame()

for i in range(12000):
    name = 'numbers/img' + str(i) + '.jpg'
    original_mat = np.array(Image.open(name), dtype=int)

    integral_mat = np.zeros(original_mat.shape, dtype=int)

    integral_image_done = summed_mat(original_mat)

    haar_features = []


    for f in feature_type:
        count = 0
        for width in range(f[0], frame_size, f[0]):
            for height in range(f[1], frame_size, f[1]):
                for x in range(frame_size - width):
                    for y in range(frame_size - height):
                        if count < 100:
                            haar_features.append(haar_feature_value(integral_image_done, f, (x,y), width, height))
                            count += 1
    df = pd.DataFrame([haar_features])
#     print (df)
    data = data.append(df)


In [6]:
len(haar_features)

200

In [7]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,28402,-41,-2,36,97,315,777,1316,2092,3409,...,-255,-143,0,0,-5,-7,-1,-1,-15,-28
0,31868,-32,0,39,56,100,110,766,3005,5707,...,-14,-1,0,0,0,0,0,0,0,-8
0,20367,-94,-2,47,892,3167,4780,5217,5661,6110,...,-60,-21,-4,-12,0,-62,-92,-5,-6,0
0,17667,0,0,0,0,0,0,0,0,227,...,-11,-15,0,0,0,0,0,0,0,0
0,23954,-20,0,16,41,69,112,195,1108,2499,...,-11,-15,0,0,0,0,0,0,0,-1


In [10]:
labels = list()
for i in range(12000):
    labels.append(labels_f[i][0])

In [12]:
y_df = pd.DataFrame(labels)

In [19]:
y_df

,label
0,5
1,0
2,4
3,1
4,9
5,2
6,1
7,3
8,1
9,4


In [28]:
y_df.columns = ['label']

In [34]:
data = data.drop(['labels'], axis = 1)

In [39]:
data.to_csv('X_df.csv', sep=',')

In [37]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,28402,-41,-2,36,97,315,777,1316,2092,3409,...,-255,-143,0,0,-5,-7,-1,-1,-15,-28
0,31868,-32,0,39,56,100,110,766,3005,5707,...,-14,-1,0,0,0,0,0,0,0,-8
0,20367,-94,-2,47,892,3167,4780,5217,5661,6110,...,-60,-21,-4,-12,0,-62,-92,-5,-6,0
0,17667,0,0,0,0,0,0,0,0,227,...,-11,-15,0,0,0,0,0,0,0,0
0,23954,-20,0,16,41,69,112,195,1108,2499,...,-11,-15,0,0,0,0,0,0,0,-1


In [40]:
y_df.to_csv('y_df.csv', sep=',')